In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from underthesea import word_tokenize
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping

# Nhập dữ liệu

In [2]:
# Load data
df = pd.read_csv('./data/product_df.csv')
df = df[['Star Rating', 'Comment']]

# Gắn nhãn cho bộ dữ liệu

In [3]:
def label_sentiment(rating):
    if rating in [1, 2]:
        return '0'
    elif rating == 3:
        return '1'
    elif rating in [4, 5]:
        return '2'
    else:
        return '3'  # Nếu có xếp hạng nằm ngoài khoảng 1-5

# Gắn nhãn cảm xúc cho mỗi đánh giá
df['Sentiment'] = df['Star Rating'].apply(label_sentiment)

# Hiển thị 5 hàng đầu tiên của dataframe với cột sentiment mới
print(df.head())

   Star Rating                                            Comment Sentiment
0            4                      Điện thoại này dùng rất thích         2
1            4                               sử dụng thấy cũng ok         2
2            2                      Bảo hành ít quá, chỉ 12 tháng         0
3            5                             Sản phẩm mượt, chạy êm         2
4            3  Cho mình hỏi muốn khởi động lại máy hay tắt ng...         1


# Tiền xử lý dữ liệu

In [4]:
# Kiểm tra và loại bỏ giá trị khuyết thiếu
print(df.isnull().sum())
df = df.dropna(subset=['Comment'])
# Kiểm tra và loại bỏ giá trị khuyết thiếu
print(df.isnull().sum())
df = df.dropna(subset=['Comment'])
# Kiểm tra và loại bỏ dữ liệu trùng lặp
duplicate_comments = df[df.duplicated(['Comment'])]
print("Các dòng dữ liệu trùng lặp trong cột 'Comment':")
print(duplicate_comments)
df = df.drop_duplicates(['Comment'])
print("Shape after dropping duplicates:", df.shape)

Star Rating    0
Comment        0
Sentiment      0
dtype: int64
Star Rating    0
Comment        0
Sentiment      0
dtype: int64
Các dòng dữ liệu trùng lặp trong cột 'Comment':
      Star Rating                                   Comment Sentiment
48              4                                   Rất tốt         2
76              5                                    Rất ok         2
107             2                             pin tụt nhanh         0
135             5                              sản phẩm tốt         2
173             2                             hao pin nhanh         0
177             5                              sản phẩm tốt         2
181             5                                    Rất ok         2
183             5                              sản phẩm tốt         2
192             5                               sản phẩm ok         2
205             5                               máy dùng ok         2
225             4                                     

In [5]:
# Chuẩn hóa và làm sạch văn bản
def remove_special_characters(text):
    # Loại bỏ các ký tự đặc biệt, giữ lại chữ cái, số, và các dấu câu
    return re.sub(r'[^a-zA-ZÀ-ỹà-ỹ0-9\s]', '', text)

def to_lowercase(text):
    # Chuyển đổi văn bản về chữ thường
    return text.lower()

def normalize_text(text):
    text = remove_special_characters(text)
    text = to_lowercase(text)
    return text

df['Comment'] = df['Comment'].apply(normalize_text)
print(df.head())

   Star Rating                                            Comment Sentiment
0            4                      điện thoại này dùng rất thích         2
1            4                               sử dụng thấy cũng ok         2
2            2                       bảo hành ít quá chỉ 12 tháng         0
3            5                              sản phẩm mượt chạy êm         2
4            3  cho mình hỏi muốn khởi động lại máy hay tắt ng...         1


In [6]:
# Extract the list of reviews X
X = df['Comment'].values.tolist()
# Extract the labels y
y = df['Sentiment'].values.tolist()

# Tách từ, xây dựng bộ từ vựng và mã hóa

In [7]:
#Xây dựng hàm token từ underthesea
def tokenize_and_build_vocab_vietnamese(comment):
    tokens = word_tokenize(comment, format="text")
    return tokens.split()

In [8]:
vectorizer = TfidfVectorizer(tokenizer=tokenize_and_build_vocab_vietnamese, token_pattern=None,
                              max_features=5000, ngram_range=(1, 2), max_df=0.85, min_df=5)
# Fit and transform the data
X_tfidf = vectorizer.fit_transform(X)
vocabulary = vectorizer.get_feature_names_out()

# Mã hóa cho cột nhãn

In [9]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Cân bằng dữ liệu

In [10]:
#Sử dụng SMOTE để cân bằng dữ liệu

print('Trước khi cân bằng:', Counter(y))

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

print('Sau khi cân bằng:', Counter(y_resampled))


Trước khi cân bằng: Counter({2: 832, 0: 428, 1: 238})
Sau khi cân bằng: Counter({2: 832, 0: 832, 1: 832})


In [11]:
print('Kích thước dữ liệu ban đầu:', X_tfidf.shape)
print('Kích thước dữ liệu sau khi cân bằng:', X_resampled.shape)
# In ra số lượng mẫu của từng nhãn trước và sau khi cân bằng
print('Trước khi cân bằng:', Counter(y))
print('Sau khi cân bằng:', Counter(y_resampled))

Kích thước dữ liệu ban đầu: (1498, 1200)
Kích thước dữ liệu sau khi cân bằng: (2496, 1200)
Trước khi cân bằng: Counter({2: 832, 0: 428, 1: 238})
Sau khi cân bằng: Counter({2: 832, 0: 832, 1: 832})


# Chia tập dữ liệu

In [12]:
# Chia tập dữ liệu thành tập huấn luyện và tập validation (validation + test)
X_train_temp, X_test, y_train_temp, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=42)

# Chia tập tạm thời thành tập validation và tập test cuối cùng
X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.2, random_state=42)

In [13]:
# Convert sparse matrix to array
X_train = X_train.toarray()
X_val = X_val.toarray()
X_test = X_test.toarray()

# Xây dựng mô hình LSTM

In [14]:
# Build the model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# Train the model
epochs = 10
batch_size = 64

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val),
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/10
24/24 [==============================] - 3s 29ms/step - loss: 1.0654 - accuracy: 0.4903 - val_loss: 0.9961 - val_accuracy: 0.7707
Epoch 2/10
24/24 [==============================] - 0s 9ms/step - loss: 0.8682 - accuracy: 0.8123 - val_loss: 0.7104 - val_accuracy: 0.8373
Epoch 3/10
24/24 [==============================] - 0s 9ms/step - loss: 0.5218 - accuracy: 0.8831 - val_loss: 0.4366 - val_accuracy: 0.8613
Epoch 4/10
24/24 [==============================] - 0s 12ms/step - loss: 0.2971 - accuracy: 0.9245 - val_loss: 0.3400 - val_accuracy: 0.8773
Epoch 5/10
24/24 [==============================] - 0s 13ms/step - loss: 0.1805 - accuracy: 0.9539 - val_loss: 0.3084 - val_accuracy: 0.8827
Epoch 6/10
24/24 [==============================] - 0s 12ms/step - loss: 0.1212 - accuracy: 0.9686 - val_loss: 0.2944 - val_accuracy: 0.8880
Epoch 7/10
24/24 [==============================] - 0s 10ms/step - loss: 0.0786 - accuracy: 0.9866 - val_loss: 0.2860 - val_accuracy: 0.8907
Epoch 8/10
24/2

# Kiểm thử và dự đoán mô hình

In [16]:
# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=1)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

20/20 [==============================] - 0s 3ms/step - loss: 0.2724 - accuracy: 0.9215
Test Score: 0.2723843455314636
Test Accuracy: 0.9214743375778198


In [17]:
# Định nghĩa các hàm tiền xử lý giống như trước
def remove_special_characters(text):
    return re.sub(r'[^a-zA-ZÀ-ỹà-ỹ0-9\s]', '', text)

def to_lowercase(text):
    return text.lower()

def normalize_text(text):
    text = remove_special_characters(text)
    text = to_lowercase(text)
    return text

def tokenize_and_build_vocab_vietnamese(comment):
    tokens = word_tokenize(comment, format="text")
    return tokens.split()

# Đánh giá mới cần kiểm thử
new_review = "Shop có cho đổi điện thoại không?"

# Tiền xử lý đánh giá mới
new_review = normalize_text(new_review)

# Chuyển đổi đánh giá mới sang TF-IDF
new_review_tfidf = vectorizer.transform([new_review]).toarray()

# Dự đoán nhãn cho đánh giá mới
prediction = model.predict(new_review_tfidf)
predicted_label = np.argmax(prediction, axis=1)

# Chuyển đổi nhãn dự đoán từ số sang tên
predicted_sentiment = label_encoder.inverse_transform(predicted_label)
print(f"Đánh giá: {new_review}")
print(f"Dự đoán cảm xúc: {predicted_sentiment[0]}")


1/1 [==============================] - 0s 403ms/step
Đánh giá: shop có cho đổi điện thoại không
Dự đoán cảm xúc: 1


In [18]:
# Báo cáo đánh giá mô hình
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_))

20/20 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0       0.94      0.92      0.93       209
           1       0.85      0.98      0.91       204
           2       0.99      0.87      0.93       211

    accuracy                           0.92       624
   macro avg       0.93      0.92      0.92       624
weighted avg       0.93      0.92      0.92       624

